In [5]:
#!pip install anchorpy

Python implementation the the publicly available code https://github.com/smartcontractkit/chainlink-solana/blob/1f5ccb696af9eaa00c214433d722fd2771b16d1b/pkg/monitoring/source_envelope.go#L126
    

In [3]:
import asyncio
import os
import requests
import json
import logging
from anchorpy import Provider, Wallet, Program
from solana.publickey import PublicKey
from solana.rpc.async_api import AsyncClient

RPC_Endpoint = 'https://api.mainnet-beta.solana.com'
program_id = "cjg3oHmg9uuPsP8D6g29NWvhySJkdYdAo9D25PRbKXJ"

async def get_state(address, program_id, provider_uris):
    """Fetch IDL file from the chain"""
    for index, provider_uri in enumerate(provider_uris):
        try:
            logging.info(f"Trying uri {provider_uri}")
            client = AsyncClient(provider_uri)
            provider = Provider(client, Wallet.dummy())
            program_id_key = PublicKey(program_id)
            
            program = await Program.at(program_id_key, provider)
            logging.debug(program.idl)
            
            address_state = await program.account['State'].fetch(address)
            logging.debug(address_state)
            
            await program.close()

            return address_state
        except Exception as e:
            if index < (len(provider_uris) - 1):
                logging.exception("An exception occurred. Trying another uri")
            else:
                await program.close()
                raise e

def getTokenAccountBalance(token_vault, provider_uri):
    data =   {
        "jsonrpc": "2.0",
        "id": 1,
        "method": "getTokenAccountBalance",
        "params": [
          str(token_vault),
                {
                "commitment": "finalized",
                "encoding":"jsonParsed"
              } 
        ]
      }
    headers = {"Content-Type": "application/json"}

    ret = requests.post(provider_uri, data=json.dumps(data), headers=headers)
    ret = json.loads(ret.text)
    if 'result' in ret:
        ret = ret['result']['value']["amount"]
    else:
        ret = 0
    return ret

async def getLinkAvailableForPayment(address, program_id, provider_uri):
    try:
        state = await get_state(address, program_id, [provider_uri])
    except:
        state = None
        
    remaining = None     
    if state:
        countUnpaidRounds, reimbursements = 0, 0
        for oracle in state.oracles.xs:
            numRounds = int(state.config.latest_aggregator_round_id) - int(oracle.from_round_id)
            if numRounds < 0:
                numRounds = 0

            countUnpaidRounds += int(numRounds)
            reimbursements += oracle.payment_gjuels

        linkBalance = getTokenAccountBalance(state.config.token_vault, provider_uri)
        logging.info(f'linkBalance {linkBalance}')
        if linkBalance:
            amountDue = (state.config.billing.observation_payment_gjuels)*countUnpaidRounds + reimbursements
            logging.info(f'amountDue {amountDue}')
            remaining = int(linkBalance) - amountDue  
            
    return remaining


In [4]:
if __name__ == '__main__':
    logging.basicConfig(
                format='%(asctime)s %(funcName)s %(levelname)s %(message)s',
                level=os.environ.get('LOGLEVEL', 'INFO').upper(),
                datefmt='%Y-%m-%d %H:%M:a%S',
                )
    LinkAvailableForPayment = await getLinkAvailableForPayment(
                                    "6Ho7WLD9QiZLy25Te67wxTyP1C68FzumNa1ZuUKafYad",
                                     program_id,
                                     RPC_Endpoint, 
                                    )
    logging.info(f'LinkAvailableForPayment {LinkAvailableForPayment}')

2022-05-26 09:13:a51 get_state INFO Trying uri https://api.mainnet-beta.solana.com
2022-05-26 09:13:a53 getLinkAvailableForPayment INFO linkBalance 1100010001000
2022-05-26 09:13:a53 getLinkAvailableForPayment INFO amountDue 235676888353831000
2022-05-26 09:13:a53 <module> INFO LinkAvailableForPayment -235675788343830000
